In [2]:
pip install python-pptx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 12.4 MB/s eta 0:00:00


In [3]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from prophet import Prophet

# Ensure pptx directory exists
os.makedirs("/content/pptx", exist_ok=True)

# ========== Prompt for Coin Name ==========
user_input = input("Enter coin keyword (e.g., 'Binance', 'AVAX', 'CoinW'): ").strip().lower()

# Find matching CSV file
search_dir = "/content/Time Series Crypto/"
csv_files = glob.glob(os.path.join(search_dir, "*.csv"))
matched_file = None
for f in csv_files:
    if user_input in f.lower():
        matched_file = f
        break

if not matched_file:
    raise FileNotFoundError(f"No file found for keyword '{user_input}' in {search_dir}")

coin_name = os.path.basename(matched_file).replace(" Historical Data.csv", "")
print(f"✅ Using file: {matched_file}")

# ========== Helper Functions ==========
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / (avg_loss + 1e-6)
    return 100 - (100 / (1 + rs))

def parse_volume(val):
    if isinstance(val, str):
        val = val.replace(',', '').strip()
        if val.endswith('K'):
            return float(val[:-1]) * 1e3
        elif val.endswith('M'):
            return float(val[:-1]) * 1e6
        elif val.endswith('B'):
            return float(val[:-1]) * 1e9
        elif val == '-':
            return 0.0
        else:
            try:
                return float(val)
            except:
                return np.nan
    return val

# ========== Load and Clean Data ==========
df = pd.read_csv(matched_file)
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

for col in ['Price', 'Open', 'High', 'Low']:
    df[col] = df[col].replace({'\$': '', ',': ''}, regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')

if 'Vol.' in df.columns:
    df['Volume'] = df['Vol.'].apply(parse_volume)
else:
    df['Volume'] = np.nan

if 'Change %' in df.columns:
    df['ChangePct'] = df['Change %'].str.replace('%', '').str.replace(',', '.').astype(float)
else:
    df['ChangePct'] = np.nan

df = df[['Date', 'Price', 'Open', 'High', 'Low', 'Volume', 'ChangePct']].dropna(subset=['Price'])

# ========== Feature Engineering ==========
df['Return'] = df['Price'].pct_change()
df['LogReturn'] = np.log(df['Price'] / df['Price'].shift(1))
df['MA_7'] = df['Price'].rolling(7).mean()
df['MA_30'] = df['Price'].rolling(30).mean()
df['STD_7'] = df['Price'].rolling(7).std()
df['Momentum_10'] = df['Price'] - df['Price'].shift(10)
df['Volatility_7d'] = df['Return'].rolling(7).std()
df['Price_Range'] = df['High'] - df['Low']
df['Candle_Body'] = abs(df['Price'] - df['Open'])
df['Upper_Shadow'] = df['High'] - df[['Price', 'Open']].max(axis=1)
df['Lower_Shadow'] = df[['Price', 'Open']].min(axis=1) - df['Low']
df['Volume_Change'] = df['Volume'].pct_change()
df['Volatility_to_Volume'] = df['Volatility_7d'] / (df['Volume'] + 1e-6)
df['RSI_14'] = compute_rsi(df['Price'], 14)
df['Lag_1'] = df['Price'].shift(1)
df['Lag_7'] = df['Price'].shift(7)
df['Price_to_MA7'] = df['Price'] / (df['MA_7'] + 1e-6)
df['Momentum_to_Volatility'] = df['Momentum_10'] / (df['Volatility_7d'] + 1e-6)

# ========== Forecasting with Prophet ==========
FUTURE_DAYS = 100
df_prophet = df[['Date', 'Price']].dropna().rename(columns={'Date': 'ds', 'Price': 'y'})
model = Prophet(daily_seasonality=True)
model.fit(df_prophet)
future = model.make_future_dataframe(periods=FUTURE_DAYS)
forecast = model.predict(future)

# ========== Risk Analysis ==========
recent_volatility = df['Volatility_7d'].iloc[-30:].mean()
if recent_volatility > 0.05:
    volatility_comment = "⚠️ High volatility detected in the past month."
else:
    volatility_comment = "✅ Volatility is within a moderate range."

latest_rsi = df['RSI_14'].iloc[-1]
if latest_rsi > 70:
    rsi_comment = "⚠️ RSI indicates potential overbought conditions."
elif latest_rsi < 30:
    rsi_comment = "⚠️ RSI indicates potential oversold conditions."
else:
    rsi_comment = "✅ RSI is in a neutral state."

forecast_range = forecast[['yhat_upper', 'yhat_lower']].tail(FUTURE_DAYS)
avg_uncertainty = (forecast_range['yhat_upper'] - forecast_range['yhat_lower']).mean()
if avg_uncertainty > df['Price'].mean() * 0.2:
    forecast_comment = "⚠️ Forecast has wide uncertainty bands."
else:
    forecast_comment = "✅ Forecast shows reasonable confidence intervals."

# ========== Generate PowerPoint ==========
ppt = Presentation()
ppt.slide_width = Inches(11)
ppt.slide_height = Inches(8.5)

# Add Title Slide with custom layout (blank)
def add_title_slide(title, subtitle):
    slide = ppt.slides.add_slide(ppt.slide_layouts[6])  # Blank slide layout (no placeholder)

    # Add Title Textbox
    title_shape = slide.shapes.add_textbox(Inches(0.5), Inches(1), Inches(10), Inches(2))
    tf = title_shape.text_frame
    p = tf.paragraphs[0]
    p.text = title
    p.font.size = Pt(40)
    p.font.bold = True

    # Add Subtitle Textbox
    subtitle_shape = slide.shapes.add_textbox(Inches(0.5), Inches(2.5), Inches(10), Inches(1))
    subtitle_tf = subtitle_shape.text_frame
    subtitle_p = subtitle_tf.paragraphs[0]
    subtitle_p.text = subtitle
    subtitle_p.font.size = Pt(20)

# Title Slide
add_title_slide(f"Crypto Forecast Report: {coin_name}", f"Forecast Horizon: {FUTURE_DAYS} Days")

# Risk Assessment Slide
slide = ppt.slides.add_slide(ppt.slide_layouts[6])  # Blank slide
shape = slide.shapes.add_textbox(Inches(0.5), Inches(1), Inches(10), Inches(5))
tf = shape.text_frame
tf.text = "📋 Risk Assessment Summary\n"
for comment in [volatility_comment, rsi_comment, forecast_comment]:
    p = tf.add_paragraph()
    p.text = f"• {comment}"
    p.font.size = Pt(16)

# Forecast Plot Slide
fig1 = model.plot(forecast)
forecast_plot = '/content/forecast_plot.png'
plt.title('Forecast Price with Confidence Interval')
plt.savefig(forecast_plot)
plt.close()

slide = ppt.slides.add_slide(ppt.slide_layouts[6])  # Blank slide
slide.shapes.add_picture(forecast_plot, Inches(1), Inches(1.5), width=Inches(9))

# Key Indicators Slide
fig2, axs = plt.subplots(2, 2, figsize=(10, 6))
axs[0, 0].plot(df['Date'], df['Price']); axs[0, 0].set_title("Price")
axs[0, 1].plot(df['Date'], df['Volatility_7d']); axs[0, 1].set_title("Volatility (7d)")
axs[1, 0].plot(df['Date'], df['RSI_14']); axs[1, 0].set_title("RSI 14")
axs[1, 1].plot(df['Date'], df['Momentum_10']); axs[1, 1].set_title("Momentum 10")
plt.tight_layout()
indicators_plot = '/content/indicators.png'
plt.savefig(indicators_plot)
plt.close()

slide = ppt.slides.add_slide(ppt.slide_layouts[6])  # Blank slide
slide.shapes.add_picture(indicators_plot, Inches(1), Inches(1.5), width=Inches(9))

# Forecast Table Slide
slide = ppt.slides.add_slide(ppt.slide_layouts[6])  # Blank slide
shape = slide.shapes.add_textbox(Inches(0.5), Inches(1), Inches(10), Inches(6))
tf = shape.text_frame
tf.text = "📊 Forecast (Last 10 Days):"
last_forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10)
for _, row in last_forecast.iterrows():
    p = tf.add_paragraph()
    p.text = f"{row['ds'].date()}: {row['yhat']:.2f} (Low: {row['yhat_lower']:.2f}, High: {row['yhat_upper']:.2f})"
    p.font.size = Pt(12)

# Save Presentation
output_path = f"/content/pptx/{coin_name.replace(' ', '_')}_report.pptx"
ppt.save(output_path)
print(f"✅ Report saved to: {output_path}")


Enter coin keyword (e.g., 'Binance', 'AVAX', 'CoinW'): CoinW
✅ Using file: /content/Time Series Crypto/AVAX_USD CoinW Historical Data.csv


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmprmqwb9to/j_jsng2x.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmprmqwb9to/m5cd5ht6.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=37298', 'data', 'file=/tmp/tmprmqwb9to/j_jsng2x.json', 'init=/tmp/tmprmqwb9to/m5cd5ht6.json', 'output', 'file=/tmp/tmprmqwb9to/prophet_modelof88f8e6/prophet_model-20250505193305.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:33:05 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:33:05 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


✅ Report saved to: /content/pptx/AVAX_USD_CoinW_report.pptx
